In [30]:
import random
import numpy as np
import pandas as pd
import itertools

#this function is used in order to chnange every number's type in a list from str to int
def maybe_float(s):
    try:
        return float(s)
    except (ValueError, TypeError):
        return s

f = open("iris.data", "r")
data = f.read().split("\n")
data = data[:-2]
data = [x.split(",") for x in data]
int_data = []
for element in data:
    int_data.append([maybe_float(x) for x in element])
random.shuffle(int_data)
training_data = int_data[:int(0.7*len(data))]
test_data = int_data[int(0.7*len(data)):]


headers = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
training_data = pd.DataFrame(training_data,columns=headers)

#seprating training_data by class
Iris_setosa_train = training_data.loc[training_data['class']=='Iris-setosa']
Iris_versicolor_train = training_data.loc[training_data['class']=='Iris-versicolor']
Iris_virginica_train = training_data.loc[training_data['class']=='Iris-virginica']

In [31]:
def univariate_normal(x, mean, variance):
    """pdf of the univariate normal distribution."""
    return ((1. / np.sqrt(2 * np.pi * variance)) * 
            np.exp(-(x - mean)**2 / (2 * variance)))


def multivariate_normal(x, d, mean, covariance):
    """pdf of the multivariate normal distribution."""
    x_m = x - mean
    return (1. / (np.sqrt((2 * np.pi)**d * np.linalg.det(covariance))) * 
            np.exp(-(np.linalg.solve(covariance, x_m).T.dot(x_m)) / 2))

def accuracy(predictions, ground_truth):
    number_of_currect_predictions = 0
    for i in range(len(predictions)):
        if predictions[i] == ground_truth[i]:
            number_of_currect_predictions +=1
    return number_of_currect_predictions/len(predictions)

## Section A 70% training 30% test

In [32]:
#training parameters
mean_Iris_setosa = Iris_setosa_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
mean_Iris_versicolor = Iris_versicolor_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
mean_Iris_virginica = Iris_virginica_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
 
      
covariance_matrix_Iris_setosa = Iris_setosa_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].cov().values
covariance_matrix_Iris_versicolor = Iris_versicolor_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].cov().values
covariance_matrix_Iris_virginica = Iris_virginica_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].cov().values

labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
ground_truth = [x[-1] for x in test_data]
predictions = []
for t_data in test_data:
    x = t_data[:-1]
    setosa_prob = multivariate_normal(x, 4, mean_Iris_setosa, covariance_matrix_Iris_setosa)
    versicolor_prob = multivariate_normal(x, 4, mean_Iris_versicolor, covariance_matrix_Iris_versicolor)
    virginica_prob = multivariate_normal(x, 4, mean_Iris_virginica, covariance_matrix_Iris_virginica)
    prob_list = [setosa_prob, versicolor_prob, virginica_prob]
    predicted_label = prob_list.index(max(prob_list))
    predictions.append(labels[predicted_label])

accuracy_value = accuracy(predictions, ground_truth)
print(accuracy_value)

0.9555555555555556


# Section B 4-fold cross validation

In [33]:
chunks = [int_data[x:x+38] for x in range(0, len(int_data), 38)]
accuracies = []
for chunk in chunks:
    test_data = chunk
    chunks_copy = chunks.copy()
    chunks_copy.remove(test_data)
    training_data = list(itertools.chain(*chunks_copy))
    
    headers = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    training_data = pd.DataFrame(training_data,columns=headers)


    #seprating training_data by class
    Iris_setosa_train = training_data.loc[training_data['class']=='Iris-setosa']
    Iris_versicolor_train = training_data.loc[training_data['class']=='Iris-versicolor']
    Iris_virginica_train = training_data.loc[training_data['class']=='Iris-virginica']

    #training parameters
    mean_Iris_setosa = Iris_setosa_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
    mean_Iris_versicolor = Iris_versicolor_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
    mean_Iris_virginica = Iris_virginica_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values


    covariance_matrix_Iris_setosa = Iris_setosa_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].cov().values
    covariance_matrix_Iris_versicolor = Iris_versicolor_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].cov().values
    covariance_matrix_Iris_virginica = Iris_virginica_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].cov().values

    labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
    ground_truth = [x[-1] for x in test_data]
    predictions = []
    for t_data in test_data:
        x = t_data[:-1]
        setosa_prob = multivariate_normal(x, 4, mean_Iris_setosa, covariance_matrix_Iris_setosa)
        versicolor_prob = multivariate_normal(x, 4, mean_Iris_versicolor, covariance_matrix_Iris_versicolor)
        virginica_prob = multivariate_normal(x, 4, mean_Iris_virginica, covariance_matrix_Iris_virginica)
        prob_list = [setosa_prob, versicolor_prob, virginica_prob]
        predicted_label = prob_list.index(max(prob_list))
        predictions.append(labels[predicted_label])

    accuracy_value = accuracy(predictions, ground_truth)
    accuracies.append(accuracy_value)
    
print(accuracies,'mean:',sum(accuracies)/len(accuracies))

[0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9444444444444444] mean: 0.9663742690058479


# Naive Bayes on Section A

In [34]:
training_data = int_data[:int(0.7*len(data))]
test_data = int_data[int(0.7*len(data)):]
labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']


headers = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
training_data = pd.DataFrame(training_data, columns=headers)

#seprating training_data by class
Iris_setosa_train = training_data.loc[training_data['class']=='Iris-setosa']
Iris_versicolor_train = training_data.loc[training_data['class']=='Iris-versicolor']
Iris_virginica_train = training_data.loc[training_data['class']=='Iris-virginica']
all_classes = [Iris_setosa_train, Iris_versicolor_train, Iris_virginica_train]

ground_truth = [x[-1] for x in test_data]
predictions = []

# naive bayes
for t_data in test_data:
    x = t_data[:-1]
    prob_list = []
    for each_class in all_classes:
        sepal_length_liklihood =  each_class[each_class.sepal_length == x[0]].shape[0]/len(each_class)
        sepal_width_liklihood =  each_class[each_class.sepal_length == x[1]].shape[0]/len(each_class)
        petal_length_liklihood =  each_class[each_class.sepal_length == x[2]].shape[0]/len(each_class)
        petal_width_liklihood =  each_class[each_class.sepal_length == x[3]].shape[0]/len(each_class)
        probablity = sepal_length_liklihood * sepal_width_liklihood * petal_length_liklihood * petal_width_liklihood * len(each_class)/len(int_data)
        prob_list.append(probablity)
    predicted_label = prob_list.index(max(prob_list))
    predictions.append(labels[predicted_label])
        
print(accuracy(predictions, ground_truth))

0.4


# Naive Bayes with 4-fold validation

In [35]:
chunks = [int_data[x:x+38] for x in range(0, len(int_data), 38)]
accuracies = []
for chunk in chunks:
    test_data = chunk
    chunks_copy = chunks.copy()
    chunks_copy.remove(test_data)
    training_data = list(itertools.chain(*chunks_copy))
    
    headers = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    training_data = pd.DataFrame(training_data,columns=headers)


    #seprating training_data by class
    Iris_setosa_train = training_data.loc[training_data['class']=='Iris-setosa']
    Iris_versicolor_train = training_data.loc[training_data['class']=='Iris-versicolor']
    Iris_virginica_train = training_data.loc[training_data['class']=='Iris-virginica']
    all_classes = [Iris_setosa_train, Iris_versicolor_train, Iris_virginica_train]

    ground_truth = [x[-1] for x in test_data]
    predictions = []

    # naive bayes
    for t_data in test_data:
        x = t_data[:-1]
        prob_list = []
        for each_class in all_classes:
            sepal_length_liklihood =  each_class[each_class.sepal_length == x[0]].shape[0]/len(each_class)
            sepal_width_liklihood =  each_class[each_class.sepal_length == x[1]].shape[0]/len(each_class)
            petal_length_liklihood =  each_class[each_class.sepal_length == x[2]].shape[0]/len(each_class)
            petal_width_liklihood =  each_class[each_class.sepal_length == x[3]].shape[0]/len(each_class)
            probablity = sepal_length_liklihood * sepal_width_liklihood * petal_length_liklihood * petal_width_liklihood * len(each_class)/len(int_data)
            prob_list.append(probablity)
        predicted_label = prob_list.index(max(prob_list))
        predictions.append(labels[predicted_label])
        
    accuracy_value = accuracy(predictions, ground_truth)
    accuracies.append(accuracy_value)
print(accuracies,'mean:',sum(accuracies)/len(accuracies))

[0.3684210526315789, 0.23684210526315788, 0.34210526315789475, 0.3888888888888889] mean: 0.3340643274853801


# Naive Bayes with smoothing

In [36]:
training_data = int_data[:int(0.7*len(data))]
test_data = int_data[int(0.7*len(data)):]
labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']


headers = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
training_data = pd.DataFrame(training_data, columns=headers)

#seprating training_data by class
Iris_setosa_train = training_data.loc[training_data['class']=='Iris-setosa']
Iris_versicolor_train = training_data.loc[training_data['class']=='Iris-versicolor']
Iris_virginica_train = training_data.loc[training_data['class']=='Iris-virginica']
all_classes = [Iris_setosa_train, Iris_versicolor_train, Iris_virginica_train]

ground_truth = [x[-1] for x in test_data]
predictions = []

# naive bayes
for t_data in test_data:
    x = t_data[:-1]
    prob_list = []
    for each_class in all_classes:
        sepal_length_liklihood =  each_class[each_class.sepal_length == x[0]].shape[0]/len(each_class)
        
        if sepal_length_liklihood == 0 :
            sepal_length_liklihood = 1/(len(each_class)+3)
            
        sepal_width_liklihood =  each_class[each_class.sepal_length == x[1]].shape[0]/len(each_class)
        
        if sepal_width_liklihood == 0 :
            sepal_width_liklihood = 1/(len(each_class)+3)
            
        petal_length_liklihood =  each_class[each_class.sepal_length == x[2]].shape[0]/len(each_class)
        
        if petal_length_liklihood == 0 :
            petal_length_liklihood = 1/(len(each_class)+3)
            
        petal_width_liklihood =  each_class[each_class.sepal_length == x[3]].shape[0]/len(each_class)
       
        if petal_width_liklihood == 0 :
            petal_width_liklihood = 1/(len(each_class)+3)
            
        probablity = sepal_length_liklihood * sepal_width_liklihood * petal_length_liklihood * petal_width_liklihood * len(each_class)/len(int_data)
        prob_list.append(probablity)
    predicted_label = prob_list.index(max(prob_list))
    predictions.append(labels[predicted_label])
        
print(accuracy(predictions, ground_truth))

0.5333333333333333


# Naive Bayes with 4-fold validation with smoothing

In [37]:
chunks = [int_data[x:x+38] for x in range(0, len(int_data), 38)]
accuracies = []
for chunk in chunks:
    test_data = chunk
    chunks_copy = chunks.copy()
    chunks_copy.remove(test_data)
    training_data = list(itertools.chain(*chunks_copy))
    
    headers = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    training_data = pd.DataFrame(training_data,columns=headers)


    #seprating training_data by class
    Iris_setosa_train = training_data.loc[training_data['class']=='Iris-setosa']
    Iris_versicolor_train = training_data.loc[training_data['class']=='Iris-versicolor']
    Iris_virginica_train = training_data.loc[training_data['class']=='Iris-virginica']
    all_classes = [Iris_setosa_train, Iris_versicolor_train, Iris_virginica_train]

    ground_truth = [x[-1] for x in test_data]
    predictions = []

    # naive bayes
    for t_data in test_data:
        x = t_data[:-1]
        prob_list = []
        for each_class in all_classes:
            sepal_length_liklihood =  each_class[each_class.sepal_length == x[0]].shape[0]/len(each_class)

            if sepal_length_liklihood == 0 :
                sepal_length_liklihood = 1/(len(each_class)+3)

            sepal_width_liklihood =  each_class[each_class.sepal_length == x[1]].shape[0]/len(each_class)

            if sepal_width_liklihood == 0 :
                sepal_width_liklihood = 1/(len(each_class)+3)

            petal_length_liklihood =  each_class[each_class.sepal_length == x[2]].shape[0]/len(each_class)

            if petal_length_liklihood == 0 :
                petal_length_liklihood = 1/(len(each_class)+3)

            petal_width_liklihood =  each_class[each_class.sepal_length == x[3]].shape[0]/len(each_class)

            if petal_width_liklihood == 0 :
                petal_width_liklihood = 1/(len(each_class)+3)
            
            probablity = sepal_length_liklihood * sepal_width_liklihood * petal_length_liklihood * petal_width_liklihood * len(each_class)/len(int_data)
            prob_list.append(probablity)
        predicted_label = prob_list.index(max(prob_list))
        predictions.append(labels[predicted_label])
        
    accuracy_value = accuracy(predictions, ground_truth)
    accuracies.append(accuracy_value)
print(accuracies,'mean:',sum(accuracies)/len(accuracies))

[0.5263157894736842, 0.47368421052631576, 0.5263157894736842, 0.5277777777777778] mean: 0.5135233918128654


# Gaussian Naive Bayes 

In [38]:
training_data = int_data[:int(0.7*len(data))]
test_data = int_data[int(0.7*len(data)):]
labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']


headers = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
training_data = pd.DataFrame(training_data, columns=headers)

#seprating training_data by class
Iris_setosa_train = training_data.loc[training_data['class']=='Iris-setosa']
Iris_versicolor_train = training_data.loc[training_data['class']=='Iris-versicolor']
Iris_virginica_train = training_data.loc[training_data['class']=='Iris-virginica']

#training parameters
mean_Iris_setosa = Iris_setosa_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
mean_Iris_versicolor = Iris_versicolor_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
mean_Iris_virginica = Iris_virginica_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
 
      
variance_Iris_setosa = Iris_setosa_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].var().values
variance_Iris_versicolor = Iris_versicolor_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].var().values
variance_Iris_virginica = Iris_virginica_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].var().values

ground_truth = [x[-1] for x in test_data]
predictions = []
for t_data in test_data:
    x = t_data[:-1]
    setosa_prob = len(Iris_setosa_train)/len(int_data)
    versicolor_prob = len(Iris_versicolor_train)/len(int_data)
    virginica_prob = len(Iris_virginica_train)/len(int_data)
    for i in range(4):
        setosa_prob *= univariate_normal(x[i], mean_Iris_setosa[i], variance_Iris_setosa[i])
        versicolor_prob *= univariate_normal(x[i], mean_Iris_versicolor[i], variance_Iris_versicolor[i])
        virginica_prob *= univariate_normal(x[i], mean_Iris_virginica[i], variance_Iris_virginica[i])
        prob_list = [setosa_prob, versicolor_prob, virginica_prob]
    predicted_label = prob_list.index(max(prob_list))
    predictions.append(labels[predicted_label])
    
print(accuracy(predictions, ground_truth))       

1.0


# Gaussian Naive Bayes 4-fold

In [39]:
chunks = [int_data[x:x+38] for x in range(0, len(int_data), 38)]
accuracies = []
for chunk in chunks:
    test_data = chunk
    chunks_copy = chunks.copy()
    chunks_copy.remove(test_data)
    training_data = list(itertools.chain(*chunks_copy))
    
    headers = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    training_data = pd.DataFrame(training_data,columns=headers)


    #seprating training_data by class
    Iris_setosa_train = training_data.loc[training_data['class']=='Iris-setosa']
    Iris_versicolor_train = training_data.loc[training_data['class']=='Iris-versicolor']
    Iris_virginica_train = training_data.loc[training_data['class']=='Iris-virginica']

    #training parameters
    mean_Iris_setosa = Iris_setosa_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
    mean_Iris_versicolor = Iris_versicolor_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values
    mean_Iris_virginica = Iris_virginica_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].mean().values


    covariance_matrix_Iris_setosa = Iris_setosa_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].cov().values
    covariance_matrix_Iris_versicolor = Iris_versicolor_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].cov().values
    covariance_matrix_Iris_virginica = Iris_virginica_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].cov().values

    labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
    ground_truth = [x[-1] for x in test_data]
    predictions = []
    for t_data in test_data:
        x = t_data[:-1]
        setosa_prob = len(Iris_setosa_train)/len(int_data)
        versicolor_prob = len(Iris_versicolor_train)/len(int_data)
        virginica_prob = len(Iris_virginica_train)/len(int_data)
        for i in range(4):
            setosa_prob *= univariate_normal(x[i], mean_Iris_setosa[i], variance_Iris_setosa[i])
            versicolor_prob *= univariate_normal(x[i], mean_Iris_versicolor[i], variance_Iris_versicolor[i])
            virginica_prob *= univariate_normal(x[i], mean_Iris_virginica[i], variance_Iris_virginica[i])
            prob_list = [setosa_prob, versicolor_prob, virginica_prob]
        predicted_label = prob_list.index(max(prob_list))
        predictions.append(labels[predicted_label])

    accuracy_value = accuracy(predictions, ground_truth)
    accuracies.append(accuracy_value)
    
print(accuracies,'mean:',sum(accuracies)/len(accuracies))

[0.9473684210526315, 0.9473684210526315, 0.9210526315789473, 0.9722222222222222] mean: 0.9470029239766082
